# Numerical Calculus
## Differentiation
### Difference formulae

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from typing import Callable

In [ ]:
from math import factorial


def taylor_exponential(x0, h, order):
    return np.exp(x0) * np.array([
        h ** n / factorial(n)
        for n in range(0,order+1)
    ]).sum(axis=0)


fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(8,4))

x = np.linspace(-1, 1, 100)
h = np.logspace(-6,0,100)
y = np.exp(x)
ax.plot(x, y, 'k--', label="exact")

for order in range(4):
    ax.plot(x, taylor_exponential(x0=0, h=x, order=order), label=f"order={order}")
ax.legend()

In [ ]:
def polynomial(x: float) -> float:
    return x - x ** 2 + x ** 3 - x ** 4

def polynomial_derivative(x: float) -> float:
    return 1 - 2 * x + 3 * x ** 2 - 4 * x ** 3

def tangent_factory(x0: float, f: Callable[[float], float], df: Callable[[float], float]) -> Callable[[float], float]:
    y0 = f(x0)
    m = df(x0)
    c = y0 - m * x0
    
    def tangent(x: float) -> float:
        return m * x + c
    
    return tangent

In [ ]:
def first_forward_difference(f: Callable[[float], float], x: float, h: float) -> float:
    """
    first forward differences of Callable[[float], float] f at point x with step size h
    """
    ...
    

def first_backward_difference(f: Callable[[float], float], x: float, h: float) -> float:
    """
    first backward differences of Callable[[float], float] f at point x with step size h
    """
    ...
    

def first_central_difference(f: Callable[[float], float], x: float, h: float) -> float:
    """
    first central differences of Callable[[float], float] f at point x with step size h
    """
    ...

In [ ]:
x = np.linspace(0, 1, 100)
f = polynomial
df = polynomial_derivative

x0 = 0.6
tangent = tangent_factory(x0, f, df)

fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(8,4))
ax.plot(x, f(x))
ax.plot(x, tangent(x), '--', label="exact")
ax.set_xlim(0.4,0.8)
ax.set_ylim(0.25,0.35)


h = 0.1

ax.plot((x0 - h, x0 - h), (0, f(x0 - h)), color='k', linestyle='--')
ax.plot((x0, x0), (0, f(x0)), color='k', linestyle='--')
ax.plot((x0 + h, x0 + h), (0, f(x0 + h)), color='k', linestyle='--')

# ax.axline((x0, f(x0)), slope=first_forward_difference(f, x0, h), linestyle='--', color='tab:red', label="forward")
# ax.axline((x0, f(x0)), slope=first_backward_difference(f, x0, h), linestyle='--', color='tab:green', label="backward")
# ax.axline((x0-h, f(x0-h)), slope=first_central_difference(f, x0, h), linestyle='--', color='tab:purple', label="central")
# ax.axline((x0, f(x0)), slope=first_complex_difference(f, x0, h), linestyle='--', color='tab:cyan', label="complex")

ax.legend()

### Errors

In [ ]:
fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(8,4))

x0 = 0.6
df0 = df(x0)

h = np.logspace(-16,0,100)
df_approx = first_forward_difference(f, x0, h)
ax.plot(h, abs(df_approx-df0), label='first_forward')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("h")
ax.set_ylabel("error")
ax.legend()

In [ ]:
def first_complex_difference(f: Callable[[complex], complex], x: float, h: float) -> float:
    """
    first complex differences of Callable[[float], float] f at point x with step size h*1j
    """
    ...